# Financial Portfolio Manager

## Objective

Build a Financial Portfolio Manager that assists users in managing their investments using **Group Chat** for agent collaboration and **StateFlow** for dynamic workflow management based on user inputs.

This agentic system will begin with a **User Proxy Agent** which will initiate a chat with the **Group Chat Manager**.

The Group Chat Manager will then call the **Portfolio Analysis Agent**, which will take user inputs and summarize their investment portfolio.

Based on the insights from the Portfolio Analysis Agent, the workflow will be dynamically managed using StateFlow to either the **Growth Investment Agent** or the **Value Investment Agent**.

Finally, these agents will share their outputs with the **Investment Advisor Agent**, which will generate a personalized financial report for the user.

---

## Key Concepts to Use

| Concept | Description |
|---------|-------------|
| **Group Chat for Collaboration** | Agents will collaborate through a group chat managed by the Group Chat Manager |
| **Dynamic Workflow Management** | StateFlow will manage the workflow based on user inputs and agent recommendations |
| **Portfolio Analysis** | The Portfolio Analysis Agent will summarize the user's investment portfolio and determine the investment category (Growth or Value) |
| **Investment Recommendations** | The Growth Investment Agent and Value Investment Agent will provide specific investment suggestions |
| **Personalized Financial Report** | The Investment Advisor Agent will compile a detailed report based on the insights from previous agents |

---

## Expected Conversation Flow

### Step 1: User Proxy Agent

- **Role:** Initiates the chat and activates the Group Chat Manager
- **Action:** Informs the Group Chat Manager about the user's intention to manage their investments

---

### Step 2: Group Chat Manager

- **Role:** Manages the group chat and directs the conversation flow
- **Action:** Calls the Portfolio Analysis Agent to take user inputs

---

### Step 3: Portfolio Analysis Agent

- **Role:** Summarizes the user's existing portfolio and determines the investment category

- **Inputs Expected:**
  - Current salary
  - Summary of current investment portfolio (e.g., amounts in fixed deposits, SIPs, real estate, etc.)

- **Output:** Determines whether to pursue Growth or Value investments based on the user's portfolio

---

### Step 4: StateFlow Workflow Management

- **Role:** Dynamically manages the workflow based on the Portfolio Analysis Agent's recommendation
- **Action:** Directs the workflow to either the Growth Investment Agent or the Value Investment Agent

---

### Step 5: Growth Investment Agent *or* Value Investment Agent

#### Growth Investment Agent
- **Role:** Suggests investments for maximizing portfolio growth
- **Action:** Provides recommendations for high-growth investment options

#### Value Investment Agent
- **Role:** Recommends stable investments for long-term value
- **Action:** Provides recommendations for stable, long-term investment options

---

### Step 6: Investment Advisor Agent

- **Role:** Compiles a detailed report of holdings and recommendations
- **Action:** Generates a personalized financial report based on the insights from previous agents

---

## Expected Output

A personalized financial report including portfolio analysis, growth, and value investment suggestions.

> **Note:** Display the output of the Investment Advisor Agent.

In [46]:
# Import the libraries
import autogen
from autogen import ConversableAgent, GroupChat, GroupChatManager
from IPython.display import Markdown, display

In [47]:
# load env variables
from dotenv import load_dotenv
load_dotenv()

True

In [48]:
# configuration for LLM
config_list = {
    "config_list": [{"model": "gpt-4o-mini", "temperature": 0.1}]
}

In [98]:
portfolio_analyst_agent = ConversableAgent(
    "portfolio_analyst_agent",
    system_message="""You are a financial analyst. The user will provide their current salary and summary of current investments portfolio.
    You will analyze the existing portfolio and determine the investment category. 
    
    Based on your analysis, determine whether to pursue Growth or Value investments:
    - Recommend GROWTH for: younger investors, higher risk tolerance, higher income, longer investment horizon
    - Recommend VALUE for: conservative investors, lower risk tolerance, nearing retirement, capital preservation needs
    
    IMPORTANT: You MUST end your response with EXACTLY one of these two lines (no markdown, no bold):
    Investment Category: Growth
    OR
    Investment Category: Value
    """,
    llm_config=config_list,
    human_input_mode="NEVER",  
)

In [99]:
user_agent = ConversableAgent(
    "user_agent",
    system_message="""You are the user who want to get some solid investment advice from the financial analyst.
    Provide your current salary and portfolio details.""",
    human_input_mode="NEVER",  
)

In [100]:
growth_investment_agent = ConversableAgent(
    "growth_investment_agent",
    system_message="""You are an growth investment expert who Suggests investments for maximizing portfolio growth by recommending recommendation for high-growth investments options.""",
    llm_config=config_list,
    human_input_mode="NEVER", 
)

value_investment_agent = ConversableAgent(
    "value_investment_agent",
    system_message="""You are an value investment expert who Suggests recommendations for stable investments options for long-term investment growth.""",
     llm_config=config_list,
     human_input_mode="NEVER", 
)

In [101]:
investment_advisor_agent = ConversableAgent(
    name="investment_advisor_agent",
    system_message="""You are a financial investment advisor with expertise in portfolio management and investment strategy. 
    Generates a personalized financial report based on the insights from previous agents. The report should include detailed report of holdings and recommendations""",
    llm_config=config_list,
    human_input_mode="NEVER", 
)

In [108]:
def state_transition(last_speaker, groupchat):
    
    messages = groupchat.messages
    last_message = messages[-1]['content'].lower()  # Convert to lowercase

    if last_speaker is user_agent:
        return portfolio_analyst_agent
    elif last_speaker is portfolio_analyst_agent:
        if 'growth' in last_message and 'investment category' in last_message:
            return growth_investment_agent
        elif 'value' in last_message and 'investment category' in last_message:
            return value_investment_agent
        # Default fallback - check which keyword appears last (as the conclusion)
        elif 'value' in last_message:
            return value_investment_agent
        else:
            return growth_investment_agent  # Default to growth
    elif last_speaker is growth_investment_agent:
        return investment_advisor_agent
    elif last_speaker is value_investment_agent:
        return investment_advisor_agent
    elif last_speaker is investment_advisor_agent:
        return None

In [109]:
group_chat = GroupChat(
    agents=[portfolio_analyst_agent,growth_investment_agent, value_investment_agent,investment_advisor_agent],
    messages=[],
    max_round= 4,
    speaker_selection_method=state_transition,
    select_speaker_auto_llm_config=config_list
)

In [110]:
group_chat_manager = GroupChatManager(
    groupchat=group_chat,
)

In [111]:
portfolio_details = {
    "salary": 100000,
    "portfolio": {
        "stocks": 80000,
        "bonds": 0,
        "real_estate": 70000
    }
}

# content = f"""
# User's current salary: ${portfolio_details["salary"]}
# User's current portfolio:
# - Stocks: ${portfolio_details["portfolio"]["stocks"]}
# - Bonds: ${portfolio_details["portfolio"]["bonds"]}
# - real_estate: ${portfolio_details["portfolio"]["real_estate"]}
# """

content = f"""
User's current salary: $50,000
User's current portfolio:
- Stocks: $10,000
- Bonds: $60,000
- Fixed Deposits: $30,000
- SIPs: $5,000
The user is 55 years old and planning for retirement in 5 years.
They prefer stable, low-risk investments with predictable returns.
"""

In [112]:
chat_result = user_agent.initiate_chat(
    group_chat_manager,
    message=content,
    summary_method="last_msg",
)

user_agent (to chat_manager):


User's current salary: $50,000
User's current portfolio:
- Stocks: $10,000
- Bonds: $60,000
- Fixed Deposits: $30,000
- SIPs: $5,000
The user is 55 years old and planning for retirement in 5 years.
They prefer stable, low-risk investments with predictable returns.


--------------------------------------------------------------------------------

Next speaker: portfolio_analyst_agent

portfolio_analyst_agent (to chat_manager):

Based on the provided information, the user is 55 years old and planning to retire in 5 years. Their current portfolio consists of a significant amount in bonds and fixed deposits, which indicates a preference for stability and lower risk. Additionally, the user has expressed a preference for stable, low-risk investments with predictable returns, which aligns with a conservative investment strategy.

Given these factors, it is advisable to recommend a Value investment strategy, as it focuses on capital preservation and lower risk,

In [113]:
agent_name_to_find = "investment_advisor_agent"

# Iterate through the dictionary to find the last conversation for the specified agent
for agent, interactions in group_chat_manager.chat_messages.items():
    # Filter interactions by agent name
    filtered_interactions = [i for i in interactions if i.get("name") == agent_name_to_find]
    if filtered_interactions:
        # Print the last conversation's content
        # print(filtered_interactions[-1]["content"])
        display(Markdown(filtered_interactions[-1]["content"]))
        break
else:
    print("Agent not found or no interactions available.")


### Personalized Financial Report

**Client Overview:**
- **Current Salary:** $50,000
- **Age:** 55 years
- **Retirement Timeline:** 5 years
- **Investment Preference:** Stable, low-risk investments with predictable returns

---

### Current Portfolio Holdings:
- **Stocks:** $10,000
- **Bonds:** $60,000
- **Fixed Deposits:** $30,000
- **SIPs (Systematic Investment Plans):** $5,000

**Total Portfolio Value:** $105,000

---

### Portfolio Analysis:
Your current portfolio reflects a conservative investment strategy, with a significant allocation to bonds and fixed deposits. This aligns well with your preference for stability as you approach retirement. However, there is an opportunity to optimize your portfolio for better income generation and capital preservation.

---

### Recommendations for Portfolio Adjustment:

1. **Increase Allocation to Bonds:**
   - **Action:** Shift a portion of your fixed deposits and stocks into Treasury or Municipal Bonds.
   - **Rationale:** These investments are low-risk and provide predictable returns, enhancing your portfolio's stability.

2. **Invest in Dividend-Paying Stocks:**
   - **Action:** Allocate $10,000 to blue-chip stocks known for stable earnings and dividends.
   - **Rationale:** These stocks can provide a steady income stream and potential for capital appreciation.

3. **Consider Real Estate Investment Trusts (REITs):**
   - **Action:** Allocate $5,000 to REITs focusing on stable cash flow properties.
   - **Rationale:** REITs can offer good dividend yields and diversification benefits.

4. **Explore Fixed Income Funds:**
   - **Action:** Invest in bond mutual funds or ETFs that focus on high-quality bonds.
   - **Rationale:** These funds can provide diversification and professional management of your bond investments.

5. **Utilize High-Yield Savings Accounts or CDs:**
   - **Action:** Maintain a portion of your cash in high-yield savings accounts or CDs.
   - **Rationale:** These options provide liquidity and better returns than traditional savings accounts.

6. **Consider Fixed Annuities:**
   - **Action:** Investigate fixed annuities for guaranteed income in retirement.
   - **Rationale:** Annuities can provide a reliable income stream, which is beneficial as you transition into retirement.

7. **Maintain a Cash Reserve:**
   - **Action:** Ensure you have a cash reserve of at least $5,000 for emergencies.
   - **Rationale:** This provides liquidity and peace of mind for unexpected expenses.

---

### Suggested Portfolio Allocation:
- **Bonds:** $80,000
- **Fixed Deposits:** $30,000
- **Dividend-Paying Stocks:** $10,000
- **REITs:** $5,000
- **Cash Reserve:** $5,000

**Total Portfolio Value After Adjustments:** $130,000

---

### Final Thoughts:
As you approach retirement, it is essential to prioritize capital preservation and income generation. Regularly review your portfolio and make adjustments as necessary to ensure it aligns with your risk tolerance and retirement goals. Consulting with a financial advisor can provide personalized guidance tailored to your specific situation.

Feel free to reach out if you have any questions or need further assistance with your investment strategy!